In [262]:
#
# classifer notebook
#
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Lambda, Compose
from LandmarkDataset import LandmarkDataset

#
# MODEL
#

class NN(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    
#
# PARAMS
#

batch_size = 4

input_size = 2 * (21 * 3) + 12 #138
hidden_size = 96
num_classes = 3

learning_rate = .01
num_epochs = 100


#
# INITIAL DATA
#

transformations = Compose([
    Lambda(lambda x: torch.tensor(x.values).float())
])

dataset = LandmarkDataset("/home/jovyan/train/data/landmarks_indexed.csv",
                              transform=transformations)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#
# INIT MODELS, LOSS FN, GRAD
#

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#
# TRAIN
#

running_loss_epoch = 0
for epoch in range(num_epochs):
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  

        # zero out accumulated gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(landmarks)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # NB: len(dataloader) is num of batches
        running_loss_epoch += loss.item()
        if ((epoch+1) % 10 == 0) and ((batch_idx+1) % len(dataloader) == 0):
            print ('Epoch [{}/{}], Loss: {:.4f}, Running Loss (per N epoch): {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item(), running_loss_epoch))
            running_loss_epoch = 0


Epoch [10/100], Loss: 1.2975, Running Loss (per N epoch): 80.0607
Epoch [20/100], Loss: 0.8289, Running Loss (per N epoch): 74.1509
Epoch [30/100], Loss: 1.0019, Running Loss (per N epoch): 70.0055
Epoch [40/100], Loss: 1.2610, Running Loss (per N epoch): 64.4997
Epoch [50/100], Loss: 1.2691, Running Loss (per N epoch): 55.9403
Epoch [60/100], Loss: 0.9276, Running Loss (per N epoch): 52.5517
Epoch [70/100], Loss: 1.1543, Running Loss (per N epoch): 50.0278
Epoch [80/100], Loss: 0.3985, Running Loss (per N epoch): 43.9705
Epoch [90/100], Loss: 1.1935, Running Loss (per N epoch): 43.6116
Epoch [100/100], Loss: 0.0444, Running Loss (per N epoch): 41.1312


In [267]:
#
# EVAL
#
torch.set_printoptions(precision=4, sci_mode=False)
softmax = torch.nn.Softmax(dim=1)
accuracy = 0
count = 0
with torch.no_grad():
    
    for batch_idx, (labels, landmarks) in enumerate(dataloader):  
        out = model(landmarks)
        prob = softmax(out.data)     #setup for threshold or 'garbage' class
        _, klass = torch.max(out.data, 1)
        
        print(klass, labels, klass==labels)
        print(prob)
        print("-----")
        
        accuracy += (klass == labels).sum().item()
        count += len(labels)
        

print('--------')
print("Accuracy {}/{} : {:.4f}".format(accuracy, count, accuracy/count))


tensor([1, 1, 1, 1]) tensor([2, 1, 1, 2]) tensor([False,  True,  True, False])
tensor([[0.2708, 0.4342, 0.2950],
        [0.2708, 0.4342, 0.2950],
        [0.0389, 0.7955, 0.1655],
        [0.2708, 0.4342, 0.2950]])
-----
tensor([2, 1, 1, 1]) tensor([2, 1, 1, 1]) tensor([True, True, True, True])
tensor([[    0.0001,     0.0440,     0.9559],
        [    0.3447,     0.6511,     0.0042],
        [    0.1547,     0.8240,     0.0213],
        [    0.2708,     0.4342,     0.2950]])
-----
tensor([1, 2, 1, 1]) tensor([0, 2, 0, 1]) tensor([False,  True, False,  True])
tensor([[    0.2708,     0.4342,     0.2950],
        [    0.0001,     0.0492,     0.9506],
        [    0.2708,     0.4342,     0.2950],
        [    0.2708,     0.4342,     0.2950]])
-----
tensor([1, 1, 1, 2]) tensor([1, 1, 0, 2]) tensor([ True,  True, False,  True])
tensor([[    0.0723,     0.8555,     0.0722],
        [    0.2708,     0.4342,     0.2950],
        [    0.2708,     0.4342,     0.2950],
        [    0.0001,     

In [264]:
print("Done")

Done
